In [2]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 13.0 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
print(tf.__version__)

2.12.0


In [5]:
fashion_mnist=keras.datasets.fashion_mnist

In [6]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [7]:
#scale down the images
train_images=train_images/255.0

In [8]:
test_images=test_images/255.0

In [9]:
train_images[0].shape

(28, 28)

In [10]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [11]:
def build_model(hp):
  model=keras.Sequential([
      # hp.Int creates a range of min to max kernel_size is filter size
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter',min_value=64,max_value=128,step=16),
          kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu',
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
          activation='relu'
      ),
      keras.layers.Dense(10,activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss="sparse_categorical_crossentropy",metrics=['accuracy'])
  return model


In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-12-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [14]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name="Mnist Fashion")

In [15]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.10266666859388351

Best val_accuracy So Far: 0.9143333435058594
Total elapsed time: 00h 03m 04s


In [17]:
model=tuner_search.get_best_models(num_models=1)[0]

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 80)        800       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        23072     
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 96)                1769568   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 1,794,410
Trainable params: 1,794,410
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 10s 5ms/step - loss: 0.1270 - accuracy: 0.9531 - val_loss: 0.2748 - val_accuracy: 0.9100
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0883 - accuracy: 0.9683 - val_loss: 0.2943 - val_accuracy: 0.9153
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0603 - accuracy: 0.9780 - val_loss: 0.3360 - val_accuracy: 0.9105
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0438 - accuracy: 0.9837 - val_loss: 0.4276 - val_accuracy: 0.9063
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0328 - accuracy: 0.9886 - val_loss: 0.4502 - val_accuracy: 0.9142
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0280 - accuracy: 0.9901 - val_loss: 0.4815 - val_accuracy: 0.9112
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0220 - accuracy: 0.9922 - val_loss: 0.5795 - val_accurac